# Credits

This is heavily influenced or copied from https://github.com/pytorch/tutorials

# Autograd: automatic differentiation

Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our first neural network.

The `autograd` package provides automatic differentiation for all operations on Tensors.
It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.

## 1. Tensor

`torch.Tensor` is the central class of the package. Setting the attribute `.requires_grad` to `True` will make the tensor "record" all operations on it. When you finish your computation you can call `.backward()` and have all the gradients computed automatically. The gradient for this tensor will be accumulated into the `.grad` attribute.

![autograd.Variable](../static_files/autograd-variable.png)

There’s one more class which is very important for autograd implementation - a `Function`.

`Tensor` and `Function` are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each tensor has a `.grad_fn` attribute that references a `Function` that has created the `Tensor` (except for Tensors created by the user - their `grad_fn` is `None`).

If you want to compute the derivatives, you can call `.backward()` on a Tensor. If `Tensor` is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to backward(), however if it has more elements, you need to specify a `gradient` argument that is a tensor of matching shape.

In [ ]:
import torch

Create a tensor

In [ ]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

Do a tensor operation:

In [ ]:
y = x + 2
print(y)

`y` was created as a result of an operation, so it has a `grad_fn`.

In [ ]:
print(y.grad_fn)

Do more operations on y

In [ ]:
z = y * y * 3
out = z.mean()

print(z)
print(out)

# Assignments

1. Create a Tensor that `requires_grad` of size (5, 5)
2. Sum the values in the Tensor

In [ ]:
# Solutions: Assignments (Tensor + Sum)
import torch

# 1) Create a Tensor that requires_grad of size (5, 5)
x5 = torch.randn(5, 5, requires_grad=True)
print("Tensor x5 (requires_grad=True):\n", x5)
print("requires_grad:", x5.requires_grad)

# 2) Sum the values in the Tensor
s5 = x5.sum()
print("\nSum of x5:", s5.item())

# Optional: backprop to see gradients
grad_output = torch.tensor(1.0)
s5.backward(grad_output)
print("\nGradient d(sum)/dx5 (all ones):\n", x5.grad)

## 2. Gradients

Let’s backprop now. Because `out` contains a single scalar, `out.backward()` is equivalent to `out.backward(torch.tensor([1.0]))`

In [ ]:
out.backward()

Print gradients d(out)/dx

In [ ]:
print(x.grad)

You should have a matrix of `4.5`. Let’s denote the tensor `out` with $o$.

We have:
$o = \frac{1}{4}\sum_i z_i$, $z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.

Therefore, $\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$,
hence $\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.

You can do many crazy things with autograd!

In [ ]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

In [ ]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

**Read later** \
*Documentation* \
`Tensor`: https://pytorch.org/docs/stable/tensors.html \
`Function`: http://pytorch.org/docs/autograd

# Assignments

1. Define a tensor and set `requires_grad` to `True`
3. Multiply the tensor by 2 and assign the result to a new python variable (i.e. `x = result`)
4. Sum the variable's elements and assign to a new python variable
5. Print the gradients of all the variables
6. Now perform a backward pass on the last variable (NOTE: for each new python variable that you define, call `.retain_grad()`)
7. Print all gradients again
  - what did you notice?

In [ ]:
# Solutions: Assignments (Gradients, retain_grad, backward)
import torch

# 1) Define a tensor and set requires_grad to True
t = torch.randn(4, requires_grad=True)
print("Tensor t:\n", t)
print("requires_grad:", t.requires_grad)

# Retain grad on t (leaf tensor grads are kept by default, but we show it explicitly)
t.retain_grad()

# 3) Multiply the tensor by 2 and assign the result to a new python variable
result = t * 2
result.retain_grad()  # retain grad for non-leaf tensor
x = result  # assign to new python variable (alias)
x.retain_grad()

# 4) Sum the variable's elements and assign to a new python variable
s = x.sum()
s.retain_grad()

# 5) Print the gradients of all the variables (before backward)
print("\nGradients BEFORE backward:")
print("t.grad:", t.grad)
print("result.grad:", result.grad)
print("x.grad:", x.grad)
print("s.grad:", s.grad)

# 6) Perform a backward pass on the last variable
s.backward()  

# 7) Print all gradients again
print("\nGradients AFTER backward:")
print("t.grad (should be all 2s):\n", t.grad)
print("result.grad (should be all 1s):\n", result.grad)
print("x.grad (alias of result; may mirror result.grad):\n", x.grad)
print("s.grad (scalar; gradient is None or retained value):", s.grad)

# Note: In PyTorch, non-leaf tensor grads are None unless retain_grad() is called.
# Leaf tensor (t) accumulates gradient in .grad by default.